In [208]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import datasets
from sklearn.metrics import confusion_matrix, accuracy_score
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader 
import copy

In [3]:
import xgboost as xgb
import lightgbm as lgbm
import catboost as cbt
from sklearn_rvm import EMRVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Diabetes Dataset

In [4]:
# import some data to play with
# digits = datasets.load_digits()

df = pd.read_csv('diabetes.csv')



In [5]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [6]:
# X = digits.data
# y = digits.target

X = df.iloc[:,:-1].values   ##features selection
y = df.iloc[:,-1].values    ## target selection

In [7]:
print("X shape")
print(X.shape)
print("Y shape")
print(y.shape)


X shape
(768, 8)
Y shape
(768,)


In [8]:
rand_seed = 4096
# x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=4096)
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=rand_seed, train_size=0.8)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, random_state=rand_seed, train_size=0.8)

In [9]:
scaler = MinMaxScaler()

scaler = scaler.fit(x_train)


In [10]:
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
x_valid = scaler.transform(x_valid)

In [36]:
# data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
# target = raw_df.values[1::2, 2]
# x_train, x_test, y_train, y_test = train_test_split(data, target, random_state=4096)

# SVM - Linear

In [37]:

svm_classifier = SVC(kernel='linear')
svm_classifier = svm_classifier.fit(x_train, y_train)

In [38]:
y_pred = svm_classifier.predict(x_test)



print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))


Confusion Matrix
 [[88  6]
 [22 38]]
Accuracy  0.8181818181818182


# SVM - Kernel

In [39]:
param_grid = [
    {'kernel': ['linear'], 'C': [1.0, 2.0, 3.0, 10., 30., 100., 300., 1000., 3000.]},
    {'kernel': ['rbf'], 'C': [1.0, 2.0, 3.0, 5.0, 10., 30., 100., 300., 1000.0],
    'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]},
]

In [40]:

svm_classifier = SVC(kernel='linear')
# svm_classifier = svm_classifier.fit(x_train, y_train)

grid_search = GridSearchCV(svm_classifier, param_grid, cv=7, scoring="neg_mean_squared_error", verbose=2)
best_svm_classifier = grid_search.fit(x_train, y_train)



Fitting 7 folds for each of 63 candidates, totalling 441 fits
[CV] END ...............................C=1.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=1.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=1.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=1.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=1.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=1.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=1.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=2.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=2.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=2.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=2.0, kernel=linear; total time=   0.0s
[CV] END ...............................C=2.0, 

In [41]:
grid_search.best_params_


{'C': 300.0, 'gamma': 0.03, 'kernel': 'rbf'}

In [42]:
# y_pred = svm_classifier.predict(x_test)
y_pred = best_svm_classifier.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[88  6]
 [20 40]]
Accuracy  0.8311688311688312


# Basic Deep Neural Network

In [210]:
print('Unique : ', np.unique(y_train))

num = np.unique(y_train, axis=0)

num = num.shape[0]

y_train_onehot = np.eye(num)[y_train]
y_test_onehot = np.eye(num)[y_test]
y_valid_onehot = np.eye(num)[y_valid]
# print(y_train_onehot)





Unique :  [0 1]


In [211]:
class TrainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)



class TestData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

train_data = TrainData(torch.FloatTensor(x_train), 
                       torch.FloatTensor(y_train_onehot))

test_data = TestData(torch.FloatTensor(x_test))
valid_data = TrainData(torch.FloatTensor(x_valid), torch.FloatTensor(y_valid_onehot))

In [212]:
train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)
valid_loader = DataLoader(dataset=valid_data, batch_size=2)

In [213]:
x_train.shape

(491, 8)

In [214]:
EPOCHS = 3000
BATCH_SIZE = 64
LEARNING_RATE = 0.001

In [240]:
class BasicClassification(nn.Module):
    def __init__(self) -> None:
        super(BasicClassification, self).__init__()

        self.layer_1 = nn.Linear(8, 32)
        self.layer_2 = nn.Linear(32, 4)
        self.layer_out = nn.Linear(4, 2)

        # self.actvation = nn.ReLU()
        self.actvation_1 = nn.SELU()
        self.actvation_2 = nn.SELU()
        self.dropout_1 = nn.Dropout(p=0.4)
        self.dropout_2 = nn.Dropout(p=0.1)
        self.batchnorm_1 = nn.BatchNorm1d(32)
        self.batchnorm_2 = nn.BatchNorm1d(4)
    
    def forward(self, inputs):
        x = self.actvation_1(self.layer_1(inputs))
        x = self.batchnorm_1(x)
        x = self.dropout_1(x)
        x = self.actvation_2(self.layer_2(x))
        x = self.batchnorm_2(x)
        x = self.dropout_2(x)
        x = self.layer_out(x)

        return x


In [241]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [242]:
model = BasicClassification()
model.to(device)

print(model)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

BasicClassification(
  (layer_1): Linear(in_features=8, out_features=32, bias=True)
  (layer_2): Linear(in_features=32, out_features=6, bias=True)
  (layer_out): Linear(in_features=6, out_features=2, bias=True)
  (actvation_1): SELU()
  (actvation_2): SELU()
  (dropout_1): Dropout(p=0.3, inplace=False)
  (dropout_2): Dropout(p=0.1, inplace=False)
  (batchnorm_1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm_2): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [243]:
def calc_accuracy(y_pred, y_test):
    # y_pred_tag = torch.round(torch.sigmoid(y_pred))
    probs = torch.softmax(y_pred, dim=1)
    winners = probs.argmax(dim=1)
    winners_y_test = y_test.argmax(dim=1)

    # correct_results_sum = (y_pred_tag == y_test).sum().float()
    correct_results_sum = (winners == winners_y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100, decimals=2)
    
    return acc

In [244]:
best_model  = None
best_loss_on_valid = 999999999
num_train_data = len(train_loader)
num_eval_data = len(valid_loader)

for epoch in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0

    eval_epoch_loss = 0
    eval_epoch_acc = 0

    model.train()
    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()

        y_pred = model(x_batch)

        loss = criterion(y_pred, y_batch)
        acc = calc_accuracy(y_pred, y_batch)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()
    
    if epoch % 10 == 0:
        
        with torch.no_grad():
            model.eval()
            for x, y in valid_loader:
                x = x.to(device)
                y = y.to(device)

                output = model(x)

                eval_loss = criterion(output, y)
                eval_acc = calc_accuracy(output, y)

                eval_epoch_loss += eval_loss.item()
                eval_epoch_acc += eval_acc.item()
        
        if best_loss_on_valid >= (eval_epoch_loss/num_eval_data):
            best_loss_on_valid = (eval_epoch_loss/num_eval_data)
            best_model = copy.deepcopy(model)
            print("Best Model is copied - Best Loss : ", best_loss_on_valid)
        


        print(f"Epoch {epoch+0:03}: : Loss: T_{epoch_loss/num_train_data:.3f} V_{eval_epoch_loss/num_eval_data:.3f} | Acc: T_{epoch_acc/num_train_data:.3f}) V_{eval_epoch_acc/num_eval_data:.3f}")

Best Model is copied - Best Loss :  0.5102756454819634
Epoch 010: : Loss: T_0.517 V_0.510 | Acc: T_75.623) V_75.806
Best Model is copied - Best Loss :  0.4843277585722746
Epoch 020: : Loss: T_0.510 V_0.484 | Acc: T_74.212) V_76.613
Best Model is copied - Best Loss :  0.4773157817581969
Epoch 030: : Loss: T_0.502 V_0.477 | Acc: T_74.248) V_76.613
Best Model is copied - Best Loss :  0.47217808495606145
Epoch 040: : Loss: T_0.501 V_0.472 | Acc: T_75.825) V_75.000
Best Model is copied - Best Loss :  0.47115561798695593
Epoch 050: : Loss: T_0.487 V_0.471 | Acc: T_76.430) V_75.000
Epoch 060: : Loss: T_0.482 V_0.476 | Acc: T_77.749) V_77.419
Epoch 070: : Loss: T_0.480 V_0.475 | Acc: T_74.744) V_79.839
Epoch 080: : Loss: T_0.496 V_0.479 | Acc: T_75.825) V_79.032
Best Model is copied - Best Loss :  0.46901470651068994
Epoch 090: : Loss: T_0.515 V_0.469 | Acc: T_76.448) V_79.032
Epoch 100: : Loss: T_0.487 V_0.480 | Acc: T_75.935) V_76.613
Epoch 110: : Loss: T_0.491 V_0.477 | Acc: T_76.540) V_78.

KeyboardInterrupt: 

In [ ]:

# with torch.no_grad():
# model.eval()
best_model.eval()
data = torch.from_numpy(x_test).float().to(device)
answer = torch.from_numpy(y_test_onehot).float().to(device)


# data = torch.from_numpy(x_train).float().to(device)
# answer = torch.from_numpy(y_train_onehot).float().to(device)


# data = torch.from_numpy(x_valid).float().to(device)
# answer = torch.from_numpy(y_valid_onehot).float().to(device)

output = best_model(data)
acc = calc_accuracy(output, answer)

print(acc)

tensor(79.8700, device='cuda:0')


# TabNet

In [43]:
tabnet_classifier = TabNetClassifier()
# clf = TabNetMultiTaskClassifier()
tabnet_classifier.fit(
  X_train=x_train, y_train=y_train,
  eval_set = [(x_valid, y_valid)],
  patience=1000,
  max_epochs=70,
  batch_size=32
)

c:\venv_python_3.10\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.73008 | val_0_auc: 0.62323 |  0:00:00s
epoch 1  | loss: 0.59169 | val_0_auc: 0.6899  |  0:00:01s
epoch 2  | loss: 0.55054 | val_0_auc: 0.69697 |  0:00:01s
epoch 3  | loss: 0.51438 | val_0_auc: 0.72862 |  0:00:02s
epoch 4  | loss: 0.51612 | val_0_auc: 0.75084 |  0:00:03s
epoch 5  | loss: 0.51468 | val_0_auc: 0.71313 |  0:00:03s
epoch 6  | loss: 0.51523 | val_0_auc: 0.76061 |  0:00:04s
epoch 7  | loss: 0.50797 | val_0_auc: 0.77003 |  0:00:05s
epoch 8  | loss: 0.5155  | val_0_auc: 0.77037 |  0:00:06s
epoch 9  | loss: 0.4829  | val_0_auc: 0.77003 |  0:00:06s
epoch 10 | loss: 0.48297 | val_0_auc: 0.76633 |  0:00:07s
epoch 11 | loss: 0.49844 | val_0_auc: 0.77104 |  0:00:08s
epoch 12 | loss: 0.4654  | val_0_auc: 0.78081 |  0:00:08s
epoch 13 | loss: 0.46976 | val_0_auc: 0.78788 |  0:00:09s
epoch 14 | loss: 0.49158 | val_0_auc: 0.80337 |  0:00:09s
epoch 15 | loss: 0.49415 | val_0_auc: 0.79697 |  0:00:10s
epoch 16 | loss: 0.47947 | val_0_auc: 0.72997 |  0:00:11s
epoch 17 | los

c:\venv_python_3.10\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [45]:
y_pred = tabnet_classifier.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[76 18]
 [13 47]]
Accuracy  0.7987012987012987


# XGBoost

In [51]:
num_round = 3
xgb_classifier = xgb.XGBClassifier()

xgb_parameters ={'max_depth' : [3,4,5,6] , 'n_estimators': [12,24,32], 
 'learning_rate' : [0.01,0.02,0.03],
 'gamma': [0.5, 1, 2], 'random_state':[99]}

In [68]:
# xgb_classifier=xgb_classifier.fit(x_train, y_train)

xgb_grid_search = GridSearchCV(xgb_classifier, xgb_parameters, cv=7, scoring="neg_mean_squared_error", verbose=2)
best_xgb_classifier = xgb_grid_search.fit(x_train, y_train)

Fitting 7 folds for each of 108 candidates, totalling 756 fits
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=12, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=12, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=12, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=12, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=12, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=12, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=12, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=24, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=24, random_state=99; to

In [69]:

xgb_grid_search.best_params_

{'gamma': 0.5,
 'learning_rate': 0.02,
 'max_depth': 3,
 'n_estimators': 32,
 'random_state': 99}

In [70]:
# y_pred = xgb_classifier.predict(x_test)
y_pred = best_xgb_classifier.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[84 10]
 [22 38]]
Accuracy  0.7922077922077922


# LightGBM

In [55]:
num_round = 3
lgbm_classifier = lgbm.LGBMClassifier()

In [56]:
lgbm_classifier=lgbm_classifier.fit(x_train, y_train)

In [57]:

y_pred = lgbm_classifier.predict(x_test)



print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[80 14]
 [19 41]]
Accuracy  0.7857142857142857


# CatBoost

In [71]:
num_round = 3
cbt_classifier = cbt.CatBoostClassifier()

In [72]:
cbt_parameters = {'depth'         : [3,4,5,6],
                'learning_rate' : [0.01,0.02,0.03],
                'n_estimators': [12,24,32], 
}

In [73]:
# cbt_classifier=cbt_classifier.fit(x_train, y_train)

cbt_grid_search = GridSearchCV(cbt_classifier, cbt_parameters, cv=7, scoring="neg_mean_squared_error", verbose=2)
best_cbt_classifier = cbt_grid_search.fit(x_train, y_train)

Fitting 7 folds for each of 36 candidates, totalling 252 fits
0:	learn: 0.6905582	total: 1.43ms	remaining: 15.7ms
1:	learn: 0.6884019	total: 1.96ms	remaining: 9.82ms
2:	learn: 0.6858832	total: 2.5ms	remaining: 7.52ms
3:	learn: 0.6834896	total: 2.94ms	remaining: 5.88ms
4:	learn: 0.6816395	total: 3.38ms	remaining: 4.74ms
5:	learn: 0.6794942	total: 3.94ms	remaining: 3.94ms
6:	learn: 0.6774428	total: 4.47ms	remaining: 3.19ms
7:	learn: 0.6753150	total: 4.98ms	remaining: 2.49ms
8:	learn: 0.6733470	total: 5.46ms	remaining: 1.82ms
9:	learn: 0.6711389	total: 5.9ms	remaining: 1.18ms
10:	learn: 0.6689405	total: 6.4ms	remaining: 581us
11:	learn: 0.6665780	total: 6.88ms	remaining: 0us
[CV] END .......depth=3, learning_rate=0.01, n_estimators=12; total time=   0.0s
0:	learn: 0.6905886	total: 693us	remaining: 7.62ms
1:	learn: 0.6883757	total: 1.67ms	remaining: 8.36ms
2:	learn: 0.6857362	total: 2.25ms	remaining: 6.74ms
3:	learn: 0.6835076	total: 2.79ms	remaining: 5.58ms
4:	learn: 0.6816201	total: 3.27

In [74]:
cbt_grid_search.best_params_

{'depth': 4, 'learning_rate': 0.03, 'n_estimators': 24}

In [75]:
# y_pred = cbt_classifier.predict(x_test)
y_pred = best_cbt_classifier.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[87  7]
 [22 38]]
Accuracy  0.8116883116883117


# Random Forest

In [76]:
rf_param_grid = { 
    'n_estimators': [100, 200],
    'max_features': ['auto', 'sqrt'],
    'max_depth' : [3,5,6],
    'criterion' :['gini', 'entropy']
}

In [77]:
num_round = 3
rf_classifier = RandomForestClassifier()

rf_grid_search = GridSearchCV(rf_classifier, rf_param_grid, cv=7, scoring="neg_mean_squared_error", verbose=2)
best_rf_classifier = rf_grid_search.fit(x_train, y_train)



Fitting 7 folds for each of 24 candidates, totalling 168 fits
[CV] END criterion=gini, max_depth=3, max_features=auto, n_estimators=100; total time=   0.0s
[CV] END criterion=gini, max_depth=3, max_features=auto, n_estimators=100; total time=   0.0s


c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestCla

[CV] END criterion=gini, max_depth=3, max_features=auto, n_estimators=100; total time=   0.0s
[CV] END criterion=gini, max_depth=3, max_features=auto, n_estimators=100; total time=   0.0s
[CV] END criterion=gini, max_depth=3, max_features=auto, n_estimators=100; total time=   0.0s


c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestCla

[CV] END criterion=gini, max_depth=3, max_features=auto, n_estimators=100; total time=   0.0s
[CV] END criterion=gini, max_depth=3, max_features=auto, n_estimators=100; total time=   0.0s


...

c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END criterion=entropy, max_depth=6, max_features=auto, n_estimators=200; total time=   0.1s
[CV] END criterion=entropy, max_depth=6, max_features=auto, n_estimators=200; total time=   0.1s


c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END criterion=entropy, max_depth=6, max_features=sqrt, n_estimators=100; total time=   0.0s
[CV] END criterion=entropy, max_depth=6, max_features=sqrt, n_estimators=100; total time=   0.0s
[CV] END criterion=entropy, max_depth=6, max_features=sqrt, n_estimators=100; total time=   0.0s
[CV] END criterion=entropy, max_depth=6, max_features=sqrt, n_estimators=100; total time=   0.0s
[CV] END criterion=entropy, max_depth=6, max_features=sqrt, n_estimators=100; total time=   0.0s
[CV] END criterion=entropy, max_depth=6, max_features=sqrt, n_estimators=100; total time=   0.0s
[CV] END criterion=entropy, max_depth=6, max_features=sqrt, n_estimators=100; total time=   0.0s
[CV] END criterion=entropy, max_depth=6, max_features=sqrt, n_estimators=200; total time=   0.1s
[CV] END criterion=entropy, max_depth=6, max_features=sqrt, n_estimators=200; total time=   0.1s
[CV] END criterion=entropy, max_depth=6, max_features=sqrt, n_estimators=200; total time=   0.1s
[CV] END criterion=entropy, ma

c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


In [78]:
# rf_classifier = rf_classifier.fit(x_train, y_train)

In [79]:
best_rf_classifier.best_params_

{'criterion': 'entropy',
 'max_depth': 5,
 'max_features': 'auto',
 'n_estimators': 200}

In [80]:

y_pred = best_rf_classifier.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[85  9]
 [21 39]]
Accuracy  0.8051948051948052


# RVM(Relevance Vector Machine)

In [87]:
em_rvc = EMRVC(kernel="linear")
em_rvc = em_rvc.fit(x_train, y_train)


y_pred = em_rvc.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

# print(clf.predict(x_test))
# print(clf.predict_proba(X))
# print(clf.score(X, y))

Confusion Matrix
 [[87  7]
 [19 41]]
Accuracy  0.8311688311688312


In [88]:
em_rvc = EMRVC(kernel="rbf")
em_rvc = em_rvc.fit(x_train, y_train)


y_pred = em_rvc.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

c:\venv_python_3.10\lib\site-packages\sklearn_rvm\em_rvm.py:675: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  warnings.warn("The default value of gamma will change "


Confusion Matrix
 [[88  6]
 [19 41]]
Accuracy  0.8376623376623377
